In [1]:
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection

import time
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.models import resnet18
import numpy as np
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

# manual random seed is used for dataset partitioning
# to ensure reproducible results across runs
RNG = torch.Generator().manual_seed(42)

Running on device: CUDA


In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

In [3]:
# CIFAR 10 dataset
normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
# we split held out data into test and validation set
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=normalize
)
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:12<00:00, 13188017.09it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
np.where(np.array(train_set.targets) == 1)[0].shape

(5000,)

In [5]:
# Choose random forget indecies from cars
# Index of class
class_index = 1
class_set = np.where(np.array(train_set.targets) == 1)[0]
# Percantage of whole data ( from class )
amount = 0.1 # 10 %
amount_int = class_set.shape[0] * amount

# Get indeces
forget_idx = np.random.choice(class_set, int(amount_int))

# construct indices of retain from those of the forget set
forget_mask = np.zeros(len(train_set.targets), dtype=bool)
forget_mask[forget_idx] = True
retain_idx = np.arange(forget_mask.size)[~forget_mask]

# split train set into a forget and a retain set
forget_set = torch.utils.data.Subset(train_set, forget_idx)
retain_set = torch.utils.data.Subset(train_set, retain_idx)

forget_loader = torch.utils.data.DataLoader(
    forget_set, batch_size=256, shuffle=True, num_workers=2
)
retain_loader = torch.utils.data.DataLoader(
    retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
)


In [6]:
dataloaders = {
    "train": train_loader,
    "val": test_loader
}

dataset_sizes = {"train": len(train_set), "val": len(test_set)}
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(DEVICE)
                    labels = labels.to(DEVICE)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [7]:
model_ft = resnet18(weights=None, num_classes=10)
# num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 10
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
# model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [8]:

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=15)

Epoch 0/14
----------
train Loss: 1.3727 Acc: 0.5048
val Loss: 1.2494 Acc: 0.5524

Epoch 1/14
----------
train Loss: 0.9724 Acc: 0.6551
val Loss: 0.9548 Acc: 0.6703

Epoch 2/14
----------
train Loss: 0.7868 Acc: 0.7226
val Loss: 0.9168 Acc: 0.6840

Epoch 3/14
----------
train Loss: 0.6471 Acc: 0.7715
val Loss: 0.8598 Acc: 0.7113

Epoch 4/14
----------
train Loss: 0.5444 Acc: 0.8087
val Loss: 0.8919 Acc: 0.7140

Epoch 5/14
----------
train Loss: 0.4485 Acc: 0.8413
val Loss: 0.8461 Acc: 0.7339

Epoch 6/14
----------
train Loss: 0.3629 Acc: 0.8719
val Loss: 0.8554 Acc: 0.7370

Epoch 7/14
----------
train Loss: 0.1498 Acc: 0.9538
val Loss: 0.7750 Acc: 0.7788

Epoch 8/14
----------
train Loss: 0.0778 Acc: 0.9790
val Loss: 0.8429 Acc: 0.7763

Epoch 9/14
----------
train Loss: 0.0439 Acc: 0.9905
val Loss: 0.9351 Acc: 0.7722

Epoch 10/14
----------
train Loss: 0.0247 Acc: 0.9956
val Loss: 1.0514 Acc: 0.7707

Epoch 11/14
----------
train Loss: 0.0134 Acc: 0.9979
val Loss: 1.1644 Acc: 0.7683

Ep

In [15]:
from torch.optim.lr_scheduler import CosineAnnealingLR,CosineAnnealingWarmRestarts,StepLR
def kl_loss_sym(x,y):
    kl_loss = nn.KLDivLoss(reduction='batchmean')
    return kl_loss(nn.LogSoftmax(dim=-1)(x),y)

def unlearning(
        net,
        retain_loader,
        forget_loader,
        val_loader,
):
    """Simple unlearning by finetuning."""
    print('-----------------------------------')
    epochs = 8
    retain_bs = 256
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.005,
                          momentum=0.9, weight_decay=0)
    optimizer_retain = optim.SGD(net.parameters(), lr=0.001*retain_bs/64, momentum=0.9, weight_decay=1e-2)
    ##the learning rate is associated with the batchsize we used
    optimizer_forget = optim.SGD(net.parameters(), lr=3e-4, momentum=0.9, weight_decay=0)
    total_step = int(len(forget_loader)*epochs)
    retain_ld = DataLoader(retain_loader.dataset, batch_size=retain_bs, shuffle=True)
    retain_ld4fgt = DataLoader(retain_loader.dataset, batch_size=256, shuffle=True)
    scheduler = CosineAnnealingLR(optimizer_forget, T_max=total_step, eta_min=1e-6)

    net.train()
    for sample in forget_loader: ##First Stage
        inputs, output = sample
        inputs = inputs.to(DEVICE)
        optimizer.zero_grad()
        outputs = net(inputs)
        uniform_label = torch.ones_like(outputs).to(DEVICE) / outputs.shape[1] ##uniform pseudo label
        loss = kl_loss_sym(outputs, uniform_label) ##optimize the distance between logits and pseudo labels
        loss.backward()
        optimizer.step()

    net.train()
    for ep in range(epochs): ##Second Stage
        net.train()
        for sample_forget, sample_retain in zip(forget_loader, retain_ld4fgt):##Forget Round
            t = 1.15 ##temperature coefficient
            inputs_forget,inputs_retain = sample_forget[0],sample_retain[0]
            inputs_forget, inputs_retain = inputs_forget.to(DEVICE), inputs_retain.to(DEVICE)
            optimizer_forget.zero_grad()
            outputs_forget,outputs_retain = net(inputs_forget),net(inputs_retain).detach()
            loss = (-1 * nn.LogSoftmax(dim=-1)(outputs_forget @ outputs_retain.T/t)).mean() ##Contrastive Learning loss
            loss.backward()
            optimizer_forget.step()
            scheduler.step()
        for sample in retain_ld: ##Retain Round
            inputs, labels = sample
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer_retain.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_retain.step()

    print('-----------------------------------')
    return net

In [16]:
%%time
model_ft_forget = unlearning(model_ft, retain_loader, forget_loader, test_loader)

-----------------------------------
-----------------------------------
CPU times: user 2min 11s, sys: 959 ms, total: 2min 12s
Wall time: 2min 16s


In [12]:
type(forget_loader)

torch.utils.data.dataloader.DataLoader